In [1]:
! pip install xgboost

     |██▍                             | 9.6 MB 3.0 kB/s eta 10:47:56ERROR: Exception:
Traceback (most recent call last):
  File "/home/galaxy/anaconda3/lib/python3.7/site-packages/pip/_vendor/urllib3/response.py", line 437, in _error_catcher
    yield
  File "/home/galaxy/anaconda3/lib/python3.7/site-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/home/galaxy/anaconda3/lib/python3.7/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/home/galaxy/anaconda3/lib/python3.7/http/client.py", line 447, in read
    n = self.readinto(b)
  File "/home/galaxy/anaconda3/lib/python3.7/http/client.py", line 491, in readinto
    n = self.fp.readinto(b)
  File "/home/galaxy/anaconda3/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "/home/galaxy/anaconda3/lib/python3.7/ssl.py", line 1052, in recv_into
    return self.read(nb

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
# from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# def loadDataSet():
df = pd.read_table("/home/galaxy/project/logistic_regression/new/LR_total_circRNA.txt")
matrix_list, labelMat = [], []
for class_n in [0,1]:
    df_sub = df[df['Label'] == class_n]
    labelMat += df_sub['Label'].tolist()
    df_sub = df_sub.iloc[:, 1:] # remove circPSMD1
    matrix_list.append(df_sub.fillna(df_sub.mean()))
dataMat = pd.concat(matrix_list)
X,Y = dataMat, labelMat
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y)

In [5]:
# https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0
# https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/
def run_exps(X_train:pd.DataFrame,y_train:pd.DataFrame,X_test:pd.DataFrame,y_test:pd.DataFrame)->pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
    dfs = []
    models = [
              ('LogReg', LogisticRegression()), 
              ('RF', RandomForestClassifier()),
              ('KNN', KNeighborsClassifier()),
              ('SVM', SVC()), 
              ('GNB', GaussianNB())
#               ('XGB', XGBClassifier())
            ]
    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
    target_names = ['malignant', 'benign']
    for name, model in models:
            kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
            cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
            clf = model.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            print(name)
            print(classification_report(y_test, y_pred, target_names=target_names))
            results.append(cv_results)
            names.append(name)
            this_df = pd.DataFrame(cv_results)
            this_df['model'] = name
            dfs.append(this_df)
    final = pd.concat(dfs, ignore_index=True)
    return final

res = run_exps(X_train, Y_train, X_test, Y_test)
# 

LogReg
             precision    recall  f1-score   support

  malignant       0.89      0.92      0.91        26
     benign       0.83      0.77      0.80        13

avg / total       0.87      0.87      0.87        39



/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision_weighted'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall_weighted'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib

RF
             precision    recall  f1-score   support

  malignant       0.92      0.92      0.92        26
     benign       0.85      0.85      0.85        13

avg / total       0.90      0.90      0.90        39



/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision_weighted'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall_weighted'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib

KNN
             precision    recall  f1-score   support

  malignant       0.86      0.96      0.91        26
     benign       0.90      0.69      0.78        13

avg / total       0.87      0.87      0.87        39

SVM
             precision    recall  f1-score   support

  malignant       0.93      0.96      0.94        26
     benign       0.92      0.85      0.88        13

avg / total       0.92      0.92      0.92        39

GNB
             precision    recall  f1-score   support

  malignant       0.89      0.92      0.91        26
     benign       0.83      0.77      0.80        13

avg / total       0.87      0.87      0.87        39



/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision_weighted'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall_weighted'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/galaxy/anaconda3/lib

In [10]:
# https://www.codeproject.com/Tips/5256220/How-to-Quickly-Compare-Multiple-ML-Models-on-your
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from json_config_expander import expand_configs
from xgboost import XGBClassifier
import pandas as pd

CLASSIFIER_MAPPINGS = {
   "random_forest": RandomForestClassifier,
   "xgboost": XGBClassifier,
   "logistic_regression": LogisticRegression}

BASE_CONFIG = {"classifier*": [
   {
      "name": "random_forest",
      "parameters": {"max_depth*": [3, 5], "n_estimators*": [50, 100, 200]}
   },
   {
      "name": "logistic_regression",
      "parameters": {"max_iter*": [10, 100, 1000], "C*": [0.1, 0.5, 1]}
   },
   {
      "name": "xgboost",
      "parameters": {"max_depth*": [3, 4, 5], "n_estimators*": [50, 100, 200], "learning_rate*": [0.01, 0.05, 0.1]}
   }]
}


def evaluate_model(config, X_train, X_test, y_train, y_test):
   classifier = CLASSIFIER_MAPPINGS[config["classifier"]["name"]](**config["classifier"]["parameters"])
   classifier.fit(X_train, y_train)
   test_scores = classifier.predict_proba(X_test)[:, 1]
   roc_auc = roc_auc_score(y_test, test_scores)
   return {"config": config, "roc_auc": roc_auc}


# if __name__ == '__main__':
# df = pd.read_csv('data.csv')

# X = df[[column for column in df.columns if column != 'label']]
# y = df['label']
# X_train_complete, X_test, y_train_complete, y_test = train_test_split(X, y, test_size=0.2)

results = expand_configs(BASE_CONFIG,
  lambda config: evaluate_model(config, X_train, X_test, Y_train, Y_test))

best_result = max(results, key=lambda res: res["roc_auc"])
best_config = best_result["config"]

result_on_test = evaluate_model(best_config, X_train, X_test, Y_train, Y_test)

print(f"Best config: {best_config}")
print(f"ROC AUC on Validation: {best_result['roc_auc']}")
print(f"ROC AUC on Test: {result_on_test['roc_auc']}")

ModuleNotFoundError: No module named 'xgboost'